# Take a look a counts for one sample

In [12]:
# Imports
import os
import numpy as np
from scipy.io import mmread
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from muon import atac as ac

# CellCommander
from cellcommander.qc import atac

In [19]:
# Load the dataset
adata = ac.read_10x_h5(
    "/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/MM290/outs/filtered_peak_bc_matrix.h5"
)
adata.var_names_make_unique()
adata.uns["files"] = {}
adata.uns["files"]["fragments"] = "/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/MM290/outs/fragments.tsv.gz"
adata

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 8953 × 307115
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'files'

In [48]:
# I think Python 3.9 has problem with the regex currently in the atac_qc function
import pybiomart as pbm
dataset = pbm.Dataset(name="hsapiens_gene_ensembl", host="http://www.ensembl.org")
annot = dataset.query(
        attributes=[
            "chromosome_name",
            "transcription_start_site",
            "strand",
            "external_gene_name",
            "transcript_biotype",
        ]
    )
filter = annot["Chromosome/scaffold name"].str.contains("CHR|GL|JH|MT")
annot = annot[~filter]
annot["Chromosome/scaffold name"] = "chr" + annot["Chromosome/scaffold name"]
annot.columns = ["Chromosome", "Start", "Strand", "Gene", "Transcript_type"]
annot = annot[annot["Transcript_type"] == "protein_coding"]
annot = annot[
    annot.Chromosome.isin(
        ["chr" + str(i) for i in range(1, 23)] + ["chrX", "chrY", "chrM"]
    )
]
annot.head()

,Chromosome,Start,Strand,Gene,Transcript_type
234,chrY,22490397,1,PRY,protein_coding
259,chrY,12662368,1,USP9Y,protein_coding
261,chrY,12701231,1,USP9Y,protein_coding
264,chrY,12847045,1,USP9Y,protein_coding
288,chrY,22096007,-1,PRY2,protein_coding


In [49]:
adata = atac.atac_qc(
    adata=adata,
    n_top_features=20,
    pct_counts_in_top_features_nmads=None,
    n_for_ns_calc=1000,
    ns_nmads=None,
    ns_hi=1.5,
    n_tss=1000,
    tss_annot=annot,
    tss_nmads=None,
    tss_hi=50,
    tss_low=4,
    total_counts_nmads=None,
    total_counts_low=1000,
    total_counts_hi=60000,
    n_features_nmads=None,
    n_features_low=1000,
    random_state=1234,
)

Reading Fragments:   0%|          | 0/8953000 [00:00<?, ?it/s]

Fetching Regions...: 100%|██████████| 1000/1000 [00:34<00:00, 28.97it/s]


AnnData object with n_obs × n_vars = 8953 × 307115
    obs: 'n_features_by_counts', 'total_counts', 'pct_counts_in_top_20_features', 'log_total_counts', 'nucleosome_signal', 'nuc_signal_filter', 'tss_score', 'outlier', 'atac_outlier'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'files'

In [52]:
atac.atac_qc_triplet_plot(
    adata=adata,
    outdir_path="/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/annotation/26Oct23/eda/MM290",
    output_prefix="pre",
    total_counts_bins=100,
    total_counts_low=1000,
    total_counts_hi=60000,
    tss_low=4,
    tss_hi=50,
)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:842: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  ax = sns.violinplot(
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:315: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(adata.obs[key]):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_utils.py:714: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  color = color[sort]


In [55]:
adata.obs["atac_outlier"].value_counts(), adata.obs["outlier"].value_counts()

(atac_outlier
 True     5621
 False    3332
 Name: count, dtype: int64,
 outlier
 False    8231
 True      722
 Name: count, dtype: int64)

# DONE!

---